In [ ]:
import numpy as np
import os
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import (
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario
from IPython.display import HTML, display
from matplotlib import pyplot as plt
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    ConnectPlanarSceneGraphVisualizer,
    DiagramBuilder,
    FixedOffsetFrame,
    JointIndex,
    Parser,
    PlanarJoint,
    RandomGenerator,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
)

from manipulation import ConfigureParser, running_as_notebook
from manipulation.scenarios import AddShape, ycb
from manipulation.station import MakeHardwareStation, load_scenario
from pydrake.common import temp_directory
table_top_filename = "/root/work/table_top.sdf"
table_top2_filename = "/root/work/table_top2.sdf"
table_top3_filename = "/root/work/table_top3.sdf"
banana_filename = "/root/work/banana.sdf"

In [ ]:
meshcat = StartMeshcat()
rng = np.random.default_rng(145)  # this is for python
generator = RandomGenerator(rng.integers(0, 1000))  # this is for c++

def clutter_gen():
    scenario_data = f"""
directives:
- add_model:
    name: table_top
    file: file://{table_top_filename}
- add_weld:
    parent: world
    child: table_top::table_top_center
"""
    scenario_data += f"""
- add_model:
    name: table_top2
    file: file://{table_top2_filename}
- add_weld:
    parent: world
    child: table_top2::table_top2_center
    """

    scenario_data += f"""
- add_model:
    name: table_top3
    file: file://{table_top3_filename}
- add_weld:
    parent: world
    child: table_top3::table_top3_center
    """
    for i in range(22 if running_as_notebook else 2):
        object_num = rng.integers(0, len(ycb))
        scenario_data += f"""
- add_model:
    name: thing{i}
    file: package://manipulation/hydro/{ycb[object_num]} 
"""
    if banana_filename:
        scenario_data += f"""
- add_model:
    name: your_model
    file: file://{banana_filename}
    default_free_body_pose:
        banana: 
            translation: [0, 0, 1]
            rotation: !Rpy {{ deg: [0, 0, 0] }}    
"""

    scenario = load_scenario(data=scenario_data)
    station = MakeHardwareStation(scenario, meshcat)

    simulator = Simulator(station)
    context = simulator.get_mutable_context()
    plant = station.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)

    z = 0.2
    print("plant Floating Base",set(list(plant.GetFloatingBaseBodies())[0:3]))
    print(type(plant.GetFloatingBaseBodies()))
    floating_base_bodies_list = list(plant.GetFloatingBaseBodies())
    len_bodies = len(floating_base_bodies_list)
    center_table_bodies = set(floating_base_bodies_list[0:len_bodies//3])
    for body_index in center_table_bodies:
    #for body_index in plant.GetFloatingBaseBodies():
        tf = RigidTransform(
            UniformlyRandomRotationMatrix(generator),
            [rng.uniform(-0.15, 0.15), rng.uniform(-0.15, 0.15), z],)
        plant.SetFreeBodyPose(plant_context, plant.get_body(body_index), tf)
        z += 0.1
    
    z = 0.2
    right_table_bodies = set(floating_base_bodies_list[len_bodies//3:len_bodies//3*2])
    for body_index in right_table_bodies:
        tf = RigidTransform(
            UniformlyRandomRotationMatrix(generator),
            [rng.uniform(-0.15, 0.15), 2+rng.uniform(-0.15, 0.15), z],)
        plant.SetFreeBodyPose(plant_context, plant.get_body(body_index), tf)
        z += 0.1
 
    z = 0.2
    left_table_bodies = set(floating_base_bodies_list[len_bodies//3*2:])
    for body_index in left_table_bodies:
        tf = RigidTransform(
            UniformlyRandomRotationMatrix(generator),
            [rng.uniform(-0.15, 0.15), -2+rng.uniform(-0.15, 0.15), z],)
        plant.SetFreeBodyPose(plant_context, plant.get_body(body_index), tf)
        z += 0.1

    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    #color_image = station.GetOutputPort("camera0.rgb_image").Eval(context)
    #plt.figure()
    #plt.imshow(color_image.data)
    #plt.axis("off")

clutter_gen()

INFO:drake:Meshcat listening for connections at https://3d6be65c-ca3c-46d4-a8f3-21f3622a2ace.deepnoteproject.com/7001/


Both `d` and `Tr` parameters defined for "banana2". Use the value of `d` for dissolve (line 20 in .mtl.)
Both `d` and `Tr` parameters defined for "pastry". Use the value of `d` for dissolve (line 32 in .mtl.)
==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

plant Floating Base {BodyIndex(25), BodyIndex(16), BodyIndex(24)}
<class 'set'>


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3d6be65c-ca3c-46d4-a8f3-21f3622a2ace' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>